In [2]:
import numpy as np

def estimate_patch_normal(points):
    """
    Estimate the normal vector of a 3D patch (subset of mesh vertices).
    
    Parameters:
    - points: (N, 3) numpy array of 3D coordinates

    Returns:
    - normal: (3,) unit normal vector of the best-fit plane
    """
    if points.shape[0] < 3:
        raise ValueError("At least 3 points are required to compute a plane.")

    # Step 1: Center the points
    centroid = np.mean(points, axis=0)
    centered = points - centroid

    # Step 2: SVD for best-fit plane
    _, _, vh = np.linalg.svd(centered)
    normal = vh[-1]  # last row = normal vector of plane

    # Step 3: Normalize
    return normal / np.linalg.norm(normal)


In [3]:
def estimate_interval_normals_with_interpolation(patches):
    """
    Takes a list of 10 angular patches and returns 10 normal vectors,
    interpolating missing patches from neighbors if needed.
    """
    normals = []
    for i in range(len(patches)):
        patch = patches[i]
        if patch.shape[0] >= 3:
            normals.append(estimate_patch_normal(patch))
        else:
            # Try to interpolate from neighbors
            neighbor_normals = []
            if i > 0 and patches[i-1].shape[0] >= 3:
                neighbor_normals.append(estimate_patch_normal(patches[i-1]))
            if i < len(patches)-1 and patches[i+1].shape[0] >= 3:
                neighbor_normals.append(estimate_patch_normal(patches[i+1]))
            
            if neighbor_normals:
                avg = np.mean(neighbor_normals, axis=0)
                normals.append(avg / np.linalg.norm(avg))
            else:
                # No valid neighbors — fallback zero vector or [0,0,1]
                normals.append(np.array([0.0, 0.0, 1.0]))
    
    return normals


In [4]:
import numpy as np

def estimate_patch_normal(points):
    """
    Estimate the normal vector of a 3D patch (subset of mesh vertices).

    Parameters:
    - points: (N, 3) numpy array of 3D coordinates

    Returns:
    - normal: (3,) unit normal vector of the best-fit plane
    """
    if points.shape[0] < 3:
        raise ValueError("At least 3 points are required to compute a plane.")

    # Step 1: Center the points
    centroid = np.mean(points, axis=0)
    centered = points - centroid

    # Step 2: SVD for best-fit plane
    _, _, vh = np.linalg.svd(centered)
    normal = vh[-1]  # last row = normal vector of plane

    # Step 3: Normalize
    return normal / np.linalg.norm(normal)


In [5]:
import open3d as o3d

def divide_vertically(mesh_path, thoracic_slices=10, lumbar_slices=7):
    """
    Divide the mesh into thoracic and lumbar intervals along the x-axis.

    Parameters:
    - mesh_path: str, path to the 3D mesh file
    - thoracic_slices: int, number of slices for thoracic region (default: 10)
    - lumbar_slices: int, number of slices for lumbar region (default: 7)

    Returns:
    - (thoracic_intervals, lumbar_intervals): two lists of np.arrays
    """
    total_slices = thoracic_slices + lumbar_slices

    # Load the mesh
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    vertices = np.asarray(mesh.vertices)

    # Sort vertices by X (ascending) → head is at lower X
    sorted_vertices = vertices[np.argsort(vertices[:, 0])]

    # Divide into total number of slices
    intervals = np.array_split(sorted_vertices, total_slices)

    # First slices are thoracic, last are lumbar
    thoracic_intervals = intervals[:thoracic_slices]
    lumbar_intervals = intervals[thoracic_slices:]

    return thoracic_intervals, lumbar_intervals

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
import numpy as np

def find_interval_back_patches(interval, patches_count=10):
    """
    Divide a horizontal interval (slice of mesh) into angular patches over 180° (back only).
    
    Parameters:
    - interval: (N, 3) numpy array of 3D coordinates
    - patches_count: number of angular segments (default: 10)

    Returns:
    - List of patches (each patch is a numpy array of points)
    """
    if len(interval) == 0:
        return []

    # Center X, Z (because Y is the vertical axis in this context)
    center_xz = np.mean(interval[:, [0, 2]], axis=0)
    rel = interval[:, [0, 2]] - center_xz  # (x, z) relative to center

    # Compute θ = atan2(z, x)
    theta = np.arctan2(rel[:, 1], rel[:, 0])  # [-π, π]

    # Restrict to 180 degrees: [-π/2, π/2] (i.e. back-facing half)
    back_mask = (theta >= -np.pi / 2) & (theta <= np.pi / 2)
    back_points = interval[back_mask]
    back_theta = theta[back_mask]

    # Bin by theta (remap [-π/2, π/2] → [0, 1] → bins)
    theta_normalized = (back_theta + np.pi / 2) / np.pi
    theta_bins = (theta_normalized * patches_count).astype(int)
    theta_bins = np.clip(theta_bins, 0, patches_count - 1)

    # Collect patches
    patches = [[] for _ in range(patches_count)]
    for i, point in enumerate(back_points):
        patches[theta_bins[i]].append(point)
    for i, patch in enumerate(patches):
        if len(patch) < 1:
            print(f"[WARNING] Patch {i} has only {len(patch)} points.")

    return [np.array(p) for p in patches if len(p) > 0]

In [16]:
def extract_features(mesh_path, thoracic_slices=10, lumbar_slices=7, patches_count=10):
    thoracic_intervals, lumbar_intervals = divide_vertically(mesh_path, thoracic_slices, lumbar_slices)

    thoracic_patches = []
    lumbar_patches = []

    for interval in thoracic_intervals:
        thoracic_patches.extend(find_interval_back_patches(interval, patches_count))

    for interval in lumbar_intervals:
        lumbar_patches.extend(find_interval_back_patches(interval, patches_count))

    thoracic_normals = []
    for patch in thoracic_patches:
        if patch.shape[0] >= 3:
            thoracic_normals.append(estimate_patch_normal(patch))

    lumbar_normals = []
    for patch in lumbar_patches:
        if patch.shape[0] >= 3:
            lumbar_normals.append(estimate_patch_normal(patch))

    return thoracic_normals, lumbar_normals


In [17]:
mesh_path = "mesh_filled.ply"
thoracic_normals, lumbar_normals = extract_features(mesh_path)
#normals = extract_features(mesh_path)


[WARNING] Patch 5 has only 0 points.
[WARNING] Patch 4 has only 0 points.
[WARNING] Patch 5 has only 0 points.
[WARNING] Patch 5 has only 0 points.
[WARNING] Patch 3 has only 0 points.
[WARNING] Patch 4 has only 0 points.
[WARNING] Patch 6 has only 0 points.


In [13]:
# try the function 
print(len(thoracic_normals)) # 100
print(len(lumbar_normals)) # 70

82
57


In [27]:
def list_shape(lst):
    shape = []
    while isinstance(lst, list):
        shape.append(len(lst))
        lst = lst[0] if lst else []
    return tuple(shape)

In [28]:
print(list_shape(thoracic_normals))

(69,)


In [77]:
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 12.8 MB/s eta 0:00:0000:01


In [79]:
import trimesh

# Load the mesh
mesh = trimesh.load_mesh("/kaggle/input/braceclassificationaligned/ostieod_cut_plys_aligned/train/2021 - 21 Pelinsu Özkan (Aktif).ply")

# Fill holes (Trimesh closes holes by filling boundary loops)
mesh.fill_holes()

# Export the fixed mesh
mesh.export("mesh_filled.ply")

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 9649\nproperty float x\nproperty float y\nproperty float z\nelement face 18841\nproperty list uchar int vertex_indices\nend_header\nn\x94R>[\x95\xaa=\xf1f\xb4\xbe\xa3\xe1\\>7V\xc7=\x05\xf6\xb4\xbe(Sc>\xb9p\xaa=\'_\xb6\xbe]\xdbQ>\xe9y\x8b=\x06\xbc\xb4\xbe\x1b\xc8c>\xf9e\x8b=^\xad\xb6\xbe\xb8\x14i>r\n0=1\x1a\xb5\xbe\x88\x88|>\x80\xbc\xe4=\xa15\xb6\xbe\x0cTq>\x91\xbc\xbd=m+\xb7\xbeE\xc2|>\x06\x0f\x8e=\x92\x86\xb8\xbeL%w>`\xf7^=\xd34\xb7\xbe)\xe0T>Z\xda!=]\x03\xb3\xbeo}c>\xf5\xaa\xca<\x1f\xff\xb2\xbeI\xa2w>ra\xd6<\x9a\xc7\xb3\xbe-~\x82>\xb2\x0e\xc3\xba\x97\xe2\xae\xbeD\xdfd>\x1c\xc08< \xff\xb0\xbes\x96h>\xe0\x1d\x80\xbay\x02\xaf\xbe/\xb2H>e\xcb?\xbd\x14U\xae\xbe\x05!Z>\xb4\xe4\x0e\xbd\xc7\x98\xae\xbe\xd8\x1dX>XRC\xbd\xc7\x80\xae\xbe\x8a\x18I>\xd3Y}\xbd0\r\xae\xbe|\xf3X>\xe3\xb3\x98\xbdmP\xaf\xbe\x99\x01q>e\xa3A\xbd\xb5\x1c\xaf\xbe\xe6\x18u>\xd6r\x01\xbd\xb2\xf0\xac\xbe\xb2\x7f\x80>\x16\'\x82\x